<a href="https://colab.research.google.com/github/aparokshith/MovieBuddy/blob/main/MovieBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 39.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9b2ba06cf132898a958d58e7185a1dbe99fcacf8eb1ab47229c69bb39541a233
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions
spark = SparkSession.builder.getOrCreate()

In [ ]:
#from google.colab import drive
#drive.mount('/gdrive')
#%cd /gdrive

In [6]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
! kaggle datasets download grouplens/movielens-20m-dataset

 98% 192M/195M [00:06<00:00, 36.8MB/s]
100% 195M/195M [00:06<00:00, 31.8MB/s]


In [8]:

! unzip /content/movielens-20m-dataset.zip

Archive:  /content/movielens-20m-dataset.zip
  inflating: genome_scores.csv       
  inflating: genome_tags.csv         
  inflating: link.csv                
  inflating: movie.csv               
  inflating: rating.csv              
  inflating: tag.csv                 


In [9]:
ratings_df = spark.read.csv('/content/rating.csv',header = True)
ratings_df.show( n =5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



### Data Cleaning


1.   Remove rows with no Rating
2.   Filter out movies or users with few ratings



In [10]:
ratings_df = ratings_df.filter(functions.col('rating').isNotNull())

In [11]:
# Filter out movies or users with few ratings
min_ratings = 50
movies_df = ratings_df.groupBy('movieId').count().filter(functions.col('count') >= min_ratings)
movies_df = movies_df.select('movieId')
ratings_df = ratings_df.join(movies_df, on='movieId', how='inner')
users_df = ratings_df.groupBy('userId').count().filter(functions.col('count') >= min_ratings)
users_df = users_df.select('userId')
ratings_df = ratings_df.join(users_df, on='userId', how='inner')

In [12]:
ratings_df.printSchema()
ratings_df = ratings_df.withColumn("userId", ratings_df["userId"].cast('int'))
ratings_df = ratings_df.withColumn("movieId", ratings_df["movieId"].cast('int'))
ratings_df = ratings_df.withColumn("rating", ratings_df["rating"].cast('float'))
ratings_df.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: string (nullable = true)



In [13]:
# Split the data into training and test sets
(train_df, test_df) = ratings_df.randomSplit([0.8, 0.2])
train_df = train_df.limit(100000)

In [14]:
# Define the ALS model
als = ALS(rank=50, maxIter=10, regParam=0.1, userCol='userId', itemCol='movieId', ratingCol='rating',
          coldStartStrategy='drop')

In [15]:
# Fit the ALS model to the training data
model = als.fit(train_df)